In [ ]:
# Standard library imports
import glob
import math
import time
import warnings
import subprocess²
from pathlib import Path
from typing import Dict, Literal, Optional, Tuple, Union
from dataclasses import dataclass

# Third-party scientific computing imports
import numpy as np
import pandas as pd
import scipy
import sep

# Visualization imports
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sb

# Astropy core imports
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS, utils
from astropy.table import Table
from astropy.nddata import NDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.time import Time
from astropy.modeling import models, fitting
from astropy.utils.exceptions import AstropyWarning
from astropy.visualization import (
    ImageNormalize, 
    ZScaleInterval, 
    HistEqStretch, 
    MinMaxInterval, 
    simple_norm
)
from astropy.convolution import Gaussian2DKernel, Tophat2DKernel, convolve
from astropy.samp import SAMPIntegratedClient

# Astroquery imports
from astroquery.gaia import Gaia

# Photutils imports
from photutils import detection, aperture
from photutils.utils import circular_footprint, calc_total_error
from photutils.psf import (
    EPSFBuilder,
    PSFPhotometry,
    IterativePSFPhotometry,
    extract_stars,
    make_psf_model
)
from photutils.segmentation import detect_threshold, detect_sources
from photutils.background import Background2D, MedianBackground, SExtractorBackground

# Other astronomy-specific imports
from astroplan import Observer
import astrometry
import imexam
from statsmodels import robust

# Utilities imports
from tqdm import tqdm

# Logging configuration
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Matplotlib configuration
plt.style.use('default')
plt.rcParams.update({
    'figure.figsize': (7, 5),
    'font.size': 10,
    'lines.linewidth': 2,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.titlesize': 12,
    'legend.fontsize': 8
})

In [ ]:
def query_gaia(
    ra: float, 
    dec: float, 
    focal_length: float,
    sensor_size: float,
    image_width: int,
    mag_limit: float = 19.5,
    bp_rp_min: float = -0.3,
    bp_rp_max: float = 0.9
) -> Optional[Table]:
    """
    Interroge la base de données Gaia DR3 pour obtenir des sources stellaires non-variables
    dans un champ de vue calculé à partir des paramètres optiques.

    Parameters
    ----------
    ra : float
        Ascension droite du centre du champ (degrés)
    dec : float
        Déclinaison du centre du champ (degrés)
    focal_length : float
        Longueur focale de l'instrument (mm)
    sensor_size : float
        Taille du pixel du capteur (μm)
    image_width : int
        Largeur de l'image en pixels
    mag_limit : float, optional
        Magnitude limite en bande Rp (default: 19.5)
    bp_rp_min : float, optional
        Limite inférieure de l'indice de couleur BP-RP (default: -0.3)
    bp_rp_max : float, optional
        Limite supérieure de l'indice de couleur BP-RP (default: 0.9)

    Returns
    -------
    Table or None
        Table Astropy contenant les résultats ou None en cas d'erreur

    Notes
    -----
    - Utilise la plate-scale pour calculer le rayon de recherche
    - Exclut les étoiles variables
    - Filtre sur la magnitude Rp et l'indice de couleur BP-RP
    """
    try:
        # Calcul du champ de vue en arcminutes
        plate_scale = 206.265 * sensor_size / focal_length  # arcsec/pixel
        radius = (plate_scale * image_width) / 120  # conversion en arcminutes
        
        logger.info(f"Recherche GAIA DR3 - rayon: {radius:.2f} arcmin - centre: RA={ra:.4f}°, Dec={dec:.4f}°")

        # Construction de la requête avec f-strings pour plus de clarté
        query = f"""
        SELECT 
            source_id, 
            ra, 
            dec, 
            phot_g_mean_mag,
            phot_bp_mean_mag,
            phot_rp_mean_mag,
            bp_rp,
            parallax,
            pmra,
            pmdec,
            ruwe
        FROM gaiadr3.gaia_source
        WHERE 1 = CONTAINS(
            POINT('ICRS', ra, dec),
            CIRCLE('ICRS', {ra}, {dec}, {radius}/60.)
        )
        AND phot_rp_mean_mag < {mag_limit}
        AND phot_variable_flag != 'VARIABLE'
        AND phot_rp_mean_mag IS NOT NULL
        AND bp_rp BETWEEN {bp_rp_min} AND {bp_rp_max}
        AND ruwe < 1.4  # Filtre sur la qualité astrométrique
        ORDER BY phot_rp_mean_mag ASC
        """

        # Exécution de la requête avec gestion du timeout
        job = Gaia.launch_job_async(query, timeout=300)
        results = job.get_results()
        
        if len(results) == 0:
            logger.warning("Aucune source trouvée dans le champ spécifié")
            return None
            
        logger.info(f"Nombre de sources trouvées: {len(results)}")
        return results

    except Exception as e:
        logger.error(f"Erreur lors de la requête Gaia: {str(e)}")
        return None

In [ ]:
def zscale_image(data: np.ndarray, cmap: str = 'viridis') -> None:
    """
    Affiche une image avec une échelle de couleur ajustée automatiquement en utilisant l'algorithme ZScaleInterval.

    Cette fonction détermine automatiquement les limites de l'échelle de couleur à l'aide de l'algorithme
    ZScaleInterval (couramment utilisé en astronomie), puis affiche l'image en utilisant la colormap spécifiée.

    Paramètres
    ----------
    data : np.ndarray
        Les données de l'image à afficher.
    cmap : str, optionnel
        Le nom de la colormap à utiliser pour l'affichage (par défaut 'viridis').

    Retourne
    --------
    None
        La fonction affiche l'image et n'a pas de valeur de retour.

    Exemple
    -------
    >>> import numpy as np
    >>> data = np.random.rand(100, 100)
    >>> zscale_image(data, cmap='plasma')
    """
    try:
        # Initialisation de l'algorithme ZScaleInterval pour déterminer les limites de l'échelle de couleur
        norm = ZScaleInterval()
        # Calcul des valeurs minimale et maximale pour l'échelle de couleur
        vmin, vmax = norm.get_limits(data)
        logger.info(f"Valeurs calculées - vmin: {vmin}, vmax: {vmax}")
    except Exception as e:
        logger.error(f"Erreur lors du calcul des limites de l'échelle de couleur avec ZScaleInterval: {e}")
        raise

    try:
        # Création de la figure et affichage de l'image avec la colormap spécifiée
        plt.figure()
        plt.imshow(
            data,
            vmin=vmin,        # Valeur minimale de l'échelle de couleur
            vmax=vmax,        # Valeur maximale de l'échelle de couleur
            origin='lower',   # Origine des coordonnées en bas à gauche
            cmap=cmap         # Colormap utilisée pour l'affichage
        )
        plt.colorbar(aspect=8, shrink=0.75)  # Ajout d'une barre de couleur
        logger.info("Affichage de l'image avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'affichage de l'image: {e}")
        raise

    # Affichage de l'image
    try:
        plt.show()
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de plt.show(): {e}")
        warnings.warn("Impossible d'afficher l'image avec plt.show().", RuntimeWarning)

In [ ]:
def histo_image(data: np.ndarray, cmap: str = 'viridis') -> None:
    """
    Affiche une image en appliquant une normalisation et un étirement de l'histogramme.

    Cette fonction normalise l'image à l'aide d'une normalisation basée sur MinMaxInterval et 
    applique un étirement de l'histogramme avec HistEqStretch pour mettre en valeur les détails 
    de l'image.

    Paramètres
    ----------
    data : np.ndarray
        Tableau contenant les données de l'image à afficher.
    cmap : str, optionnel
        Nom de la colormap utilisée pour l'affichage (par défaut 'viridis').

    Retourne
    --------
    None
        La fonction affiche l'image et ne retourne rien.

    Exemple
    -------
    >>> import numpy as np
    >>> data = np.random.rand(100, 100)
    >>> histo_image(data, cmap='plasma')
    """
    try:
        # Initialisation de la normalisation de l'image avec MinMaxInterval et HistEqStretch
        norm = ImageNormalize(data, 
                              interval=MinMaxInterval(),  # Normalisation basée sur les valeurs min et max
                              stretch=HistEqStretch())    # Étirement de l'histogramme égalisé
        logger.info("Normalisation de l'image initialisée avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'initialisation de la normalisation de l'image: {e}")
        raise

    try:
        # Création de la figure et affichage de l'image avec la normalisation et la colormap spécifiées
        plt.figure()
        plt.imshow(data, 
                   norm=norm,        # Application de la normalisation à l'image
                   origin='lower',   # Origine des coordonnées en bas à gauche
                   cmap=cmap)        # Colormap utilisée pour l'affichage
        plt.colorbar(aspect=8, shrink=0.75)  # Ajout d'une barre de couleur
        logger.info("Affichage de l'image effectué avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'affichage de l'image: {e}")
        raise

    try:
        plt.show()
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de plt.show(): {e}")
        raise

In [ ]:
def airmass(
    head: Dict,
    latitude: float = 48.29166,
    longitude: float = 2.43805,
    elevation: float = 95.0
) -> float:
    """
    Calcule la masse d'air pour un objet céleste à partir des informations contenues dans un header FITS.

    Paramètres
    ----------
    head : dict
        Dictionnaire contenant les informations sur les coordonnées de l'objet céleste et la date d'observation.
        Les clés attendues sont "DATE-OBS" et l'une de "RA" ou "OBJRA" pour l'ascension droite, et l'une de "DEC" ou "OBJDEC" pour la déclinaison.
    latitude : float, optionnel
        Latitude de l'observatoire en degrés (par défaut 48.29166).
    longitude : float, optionnel
        Longitude de l'observatoire en degrés (par défaut 2.43805).
    elevation : float, optionnel
        Élévation de l'observatoire en mètres (par défaut 95.0).

    Retourne
    --------
    float
        La masse d'air calculée. En cas d'erreur lors de l'extraction ou du calcul, retourne 0.0.

    Exemple
    -------
    >>> head = {
    ...     "RA": "10:00:00",
    ...     "DEC": "+10:00:00",
    ...     "DATE-OBS": "2020-01-01T00:00:00"
    ... }
    >>> am = airmass(head)
    >>> print(am)
    """
    logger = logging.getLogger(__name__)

    try:
        # Extraction des coordonnées et du temps d'observation
        ra = head.get("RA", head.get("OBJRA"))
        dec = head.get("DEC", head.get("OBJDEC"))
        obstime_str = head["DATE-OBS"]

        if ra is None or dec is None:
            raise KeyError("Les informations RA/DEC ou OBJRA/OBJDEC sont manquantes dans le header.")

        # Conversion du temps d'observation en objet Time d'Astropy
        obstime = Time(obstime_str)
    except KeyError as e:
        logger.error(f"Erreur lors de l'extraction des données du header: {e}")
        return 0.0
    except Exception as e:
        logger.error(f"Erreur inattendue lors de la conversion des données du header: {e}")
        return 0.0

    try:
        # Création de l'objet SkyCoord pour les coordonnées de l'objet
        coord = SkyCoord(ra=ra, dec=dec, unit=u.deg, frame='icrs')

        # Création de l'objet Observer pour représenter l'observatoire
        observatory = Observer(latitude=latitude*u.deg, longitude=longitude*u.deg, elevation=elevation*u.m)

        # Calcul des coordonnées alt-azimutales et de la masse d'air
        altaz = observatory.altaz(obstime, coord)
        am = altaz.secz

        if am < 1:
            logger.warning("La masse d'air est inférieure à 1, vérifiez les données d'entrée.")
    except Exception as e:
        logger.error(f"Erreur lors du calcul de la masse d'air: {e}")
        return 0.0

    logger.info(f"La masse d'air calculée est : {am}")
    return am

In [ ]:
def image_quality(
    data: np.ndarray,
    im_wcs: WCS,
    sigma: float = 3.0,
    seeing: float = 3.0
) -> Tuple[float, float, float, float, float]:
    """
    Évalue la qualité d'une image astronomique en calculant des statistiques de base 
    et en estimant la largeur à mi-hauteur (FWHM).

    Cette fonction effectue les opérations suivantes :
      - Affichage des valeurs minimale et maximale en ADU.
      - Calcul des statistiques de base (moyenne, médiane, écart-type) après sigma-clipping.
      - Calcul de l'échelle des pixels en arcsecondes à partir du WCS.
      - Estimation de la FWHM en pixels à partir du seeing (en arcsecondes).

    Paramètres
    ----------
    data : np.ndarray
        Tableau contenant les données de l'image.
    im_wcs : WCS
        Objet WCS (World Coordinate System) associé à l'image.
    sigma : float, optionnel
        Facteur de sigma utilisé pour le sigma-clipping lors du calcul des statistiques (par défaut 3.0).
    seeing : float, optionnel
        Valeur du seeing en arcsecondes pour estimer la FWHM (par défaut 3.0).

    Retourne
    --------
    Tuple[float, float, float, float, float]
        Un tuple contenant :
          - mean : Moyenne des données après sigma-clipping (en ADU).
          - median : Médiane des données après sigma-clipping (en ADU).
          - std : Écart-type des données après sigma-clipping (en ADU).
          - fwhm : Estimation de la FWHM en pixels.
          - pixel_scale : Échelle des pixels en arcsecondes.

    Exemple
    -------
    >>> from astropy.wcs import WCS
    >>> import numpy as np
    >>> # Création d'une image factice et d'un WCS fictif
    >>> data = np.random.normal(loc=1000, scale=50, size=(1024, 1024))
    >>> wcs_header = {
    ...     'CTYPE1': 'RA---TAN',
    ...     'CTYPE2': 'DEC--TAN',
    ...     'CRVAL1': 0,
    ...     'CRVAL2': 0,
    ...     'CRPIX1': 512,
    ...     'CRPIX2': 512,
    ...     'CD1_1': -0.0002777777778,
    ...     'CD1_2': 0,
    ...     'CD2_1': 0,
    ...     'CD2_2': 0.0002777777778
    ... }
    >>> im_wcs = WCS(wcs_header)
    >>> stats = image_quality(data, im_wcs, sigma=3.0, seeing=3.0)
    >>> print(stats)
    """
    try:
        # Affichage des valeurs min et max en ADU via logging
        min_val = np.min(data)
        max_val = np.max(data)
        logger.info(f"Valeur minimale: {min_val} ADU")
        logger.info(f"Valeur maximale: {max_val} ADU")
    except Exception as e:
        logger.error(f"Erreur lors du calcul des valeurs min/max: {e}")
        raise

    try:
        # Calcul des statistiques après sigma-clipping
        mean, median, std = sigma_clipped_stats(data, sigma=sigma)
        logger.info(f"Mean après sigma-clipping: {mean} ADU")
        logger.info(f"Median après sigma-clipping: {median} ADU")
        logger.info(f"StD après sigma-clipping: {std} ADU")
    except Exception as e:
        logger.error(f"Erreur lors du calcul des statistiques sigma-clipped: {e}")
        raise

    try:
        # Calcul de l'échelle des pixels en arcsecondes à partir du WCS
        scales_deg = wcs_utils.proj_plane_pixel_scales(im_wcs)
        pixel_scale = np.median(scales_deg) * u.deg.to(u.arcsec)
        logger.info(f"Pixel Scale: {round(pixel_scale, 2)} arcsec")
    except Exception as e:
        logger.error(f"Erreur lors de l'extraction de l'échelle du WCS: {e}")
        raise

    try:
        # Estimation de la FWHM en pixels à partir du seeing (en arcsec)
        fwhm = (seeing * u.arcsec) / pixel_scale
        fwhm_pixels = round(fwhm.value, 2)
        logger.info(f"FWHM estimée: {fwhm_pixels} pixels")
    except Exception as e:
        logger.error(f"Erreur lors du calcul de la FWHM: {e}")
        raise

    return mean, median, std, fwhm_pixels, round(pixel_scale, 2)


In [ ]:
def load_fits(
    file_path: Union[str, Path], 
    extension: int = 0,
    validate_wcs: bool = True,
    ignore_missing_wcs: bool = False
) -> Tuple[fits.Header, np.ndarray, Optional[WCS]]:
    """
    Charge et valide un fichier FITS astronomique.

    Parameters
    ----------
    file_path : str or Path
        Chemin vers le fichier FITS
    extension : int, optional
        Index de l'extension FITS à charger (default: 0)
    validate_wcs : bool, optional
        Valider la cohérence du WCS si présent (default: True)
    ignore_missing_wcs : bool, optional
        Ne pas lever d'avertissement si le WCS est absent (default: False)

    Returns
    -------
    header : fits.Header
        En-tête FITS de l'extension spécifiée
    data : np.ndarray
        Données de l'image
    wcs : WCS or None
        Objet WCS si les informations astrométriques sont valides, None sinon

    Raises
    ------
    FileNotFoundError
        Si le fichier n'existe pas
    IOError
        Si le fichier n'est pas un FITS valide
    ValueError
        Si l'extension demandée n'existe pas
    
    Notes
    -----
    - Vérifie la présence et la validité des informations WCS
    - Gère les cas particuliers (données vides, WCS invalide)
    - Effectue des validations basiques sur les données
    """
    try:
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"Le fichier {file_path} n'existe pas")

        # Suppression des warnings Astropy non critiques
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', category=AstropyWarning)
            
            with fits.open(file_path) as hdul:
                # Vérification de l'extension
                if extension >= len(hdul):
                    raise ValueError(f"Extension {extension} invalide. Max: {len(hdul)-1}")
                
                # Extraction des données et du header
                hdu = hdul[extension]
                header = hdu.header.copy()  # Copie pour éviter les problèmes de référence
                data = hdu.data.copy()

                # Validation basique des données
                if data is None:
                    raise ValueError("L'extension ne contient pas de données")
                
                if not isinstance(data, np.ndarray):
                    raise ValueError("Les données ne sont pas dans un format valide")

                # Vérification du WCS
                wcs_info = None
                required_wcs_keys = {
                    'CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2',
                    'CRPIX1', 'CRPIX2'
                }
                
                if all(key in header for key in required_wcs_keys):
                    try:
                        wcs_info = WCS(header)
                        
                        if validate_wcs:
                            # Validation du WCS
                            test_points = np.array([[0, 0], [data.shape[1]-1, data.shape[0]-1]])
                            sky_coords = wcs_info.all_pix2world(test_points, 0)
                            
                            # Vérification des valeurs raisonnables
                            if not (np.all(np.isfinite(sky_coords)) and 
                                  np.all((-90 <= sky_coords[:, 1]) & (sky_coords[:, 1] <= 90))):
                                logger.warning("WCS détecté mais coordonnées invalides")
                                wcs_info = None
                            
                    except Exception as e:
                        logger.warning(f"WCS détecté mais invalide: {str(e)}")
                        wcs_info = None
                elif not ignore_missing_wcs:
                    logger.warning("Pas d'information WCS dans le header")

                # Ajout d'informations statistiques au header
                with np.errstate(invalid='ignore'):
                    header['IMGMED'] = float(np.median(data))
                    header['IMGMEAN'] = float(np.mean(data))
                    header['IMGSTD'] = float(np.std(data))
                    header['IMGMIN'] = float(np.min(data))
                    header['IMGMAX'] = float(np.max(data))

                return header, data, wcs_info

    except Exception as e:
        logger.error(f"Erreur lors du chargement du fichier {file_path}: {str(e)}")
        raise

def is_calibration_frame(header: fits.Header) -> bool:
    """
    Détecte si l'image est une frame de calibration (bias, dark, flat).

    Parameters
    ----------
    header : fits.Header
        Header FITS à analyser

    Returns
    -------
    bool
        True si c'est une frame de calibration
    """
    # Mots-clés communs pour les frames de calibration
    calibration_keywords = ['BIAS', 'DARK', 'FLAT', 'SKYFLAT', 'DOMEFLAT']
    
    # Vérifie dans les mots-clés IMAGETYP, OBSTYPE, etc.
    for key in ['IMAGETYP', 'OBSTYPE', 'FRAME']:
        if key in header:
            value = header[key].upper()
            if any(cal_type in value for cal_type in calibration_keywords):
                return True
    
    return False

# try:
#     # Chargement d'une image
#     header, data, wcs = load_fits(
#         "mon_image.fits",
#         extension=0,
#         validate_wcs=True
#     )
    
#     # Vérification si c'est une frame de calibration
#     if is_calibration_frame(header):
#         logger.info("Image de calibration détectée")
        
#     # Utilisation des statistiques
#     print(f"Médiane de l'image: {header['IMGMED']}")
#     print(f"Écart-type: {header['IMGSTD']}")
    
#     if wcs is not None:
#         # Conversion de coordonnées avec le WCS
#         ra, dec = wcs.all_pix2world(100, 100, 0)
#         print(f"Coordonnées du pixel (100,100): RA={ra}, Dec={dec}")

# except Exception as e:
#     logger.error(f"Erreur: {str(e)}")

In [ ]:
def compute_background(
    img: np.ndarray,
    sigma: float = 3.0,
    box_size: Tuple[int, int] = (100, 100),
    filter_size: Tuple[int, int] = (5, 5),
    mask: Optional[np.ndarray] = None,
    exclude_percentile: float = 25.0
) -> Background2D:
    """
    Calcule le fond de ciel d'une image en utilisant la méthode `Background2D` de `photutils`.

    Paramètres
    ----------
    img : np.ndarray
        Image pour laquelle le fond de ciel doit être estimé.
    sigma : float, optionnel
        Seuil en sigma pour le sigma-clipping (par défaut 3.0).
    box_size : tuple de int, optionnel
        Taille de la boîte (en pixels) pour le calcul local du fond de ciel (par défaut (100, 100)).
    filter_size : tuple de int, optionnel
        Taille du filtre pour lisser le fond de ciel (par défaut (5, 5)).
    mask : np.ndarray, optionnel
        Masque binaire où les pixels masqués sont exclus du calcul du fond de ciel.
    exclude_percentile : float, optionnel
        Pourcentage d'exclusion utilisé par Background2D pour éviter l'influence d'outliers (par défaut 25.0).

    Retourne
    --------
    Background2D
        Objet contenant le fond de ciel estimé et la carte du bruit.

    Exemple
    -------
    >>> from photutils import Background2D
    >>> background = compute_background(image_data, sigma=3.0, box_size=(100, 100))
    >>> bkg_image = background.background
    """
    # Créer l'objet SigmaClip pour exclure les valeurs aberrantes
    sigma_clip = SigmaClip(sigma=sigma)

    # Créer l'estimateur du fond de ciel basé sur SExtractor
    bkg_estimator = SExtractorBackground()

    try:
        bkg = Background2D(
            data=img,
            box_size=box_size,
            filter_size=filter_size,
            mask=mask,
            sigma_clip=sigma_clip,
            bkg_estimator=bkg_estimator,
            exclude_percentile=exclude_percentile
        )
    except Exception as e:
        raise RuntimeError("Erreur lors du calcul du fond de ciel") from e

    return bkg

In [ ]:
def make_border_mask(
    image: np.ndarray,
    border: Union[int, Tuple[int, int], Tuple[int, int, int, int]] = 50,
    invert: bool = True,
    dtype: np.dtype = bool
) -> np.ndarray:
    """
    Crée un masque binaire pour une image en excluant une ou plusieurs bordures.

    Parameters
    ----------
    image : np.ndarray
        Image source pour laquelle le masque doit être créé
    border : int or tuple, optional
        Peut être :
        - int : même largeur pour toutes les bordures
        - tuple[int, int] : (vertical, horizontal)
        - tuple[int, int, int, int] : (haut, bas, gauche, droite)
        (default: 50)
    invert : bool, optional
        Si True, inverse le masque (bordure = True, centre = False)
        Si False, bordure = False, centre = True
        (default: True)
    dtype : np.dtype, optional
        Type de données du masque de sortie (default: bool)

    Returns
    -------
    np.ndarray
        Masque binaire de la même taille que l'image

    Raises
    ------
    ValueError
        Si les dimensions de la bordure sont invalides ou trop grandes
    TypeError
        Si les types d'entrée sont incorrects

    Examples
    --------
    >>> # Masque avec bordure uniforme de 50 pixels
    >>> mask = make_border_mask(image, 50)
    
    >>> # Masque avec bordures différentes (haut, bas, gauche, droite)
    >>> mask = make_border_mask(image, (100, 50, 75, 75))
    
    >>> # Masque non inversé (True au centre)
    >>> mask = make_border_mask(image, 50, invert=False)
    """
    # Validation du type d'entrée
    if not isinstance(image, np.ndarray):
        raise TypeError("L'image doit être un numpy.ndarray")
    
    if image.size == 0:
        raise ValueError("L'image ne peut pas être vide")

    height, width = image.shape[:2]

    # Conversion et validation des bordures
    if isinstance(border, (int, float)):
        border = int(border)
        top = bottom = left = right = border
    elif isinstance(border, tuple):
        if len(border) == 2:
            vert, horiz = border
            top = bottom = vert
            left = right = horiz
        elif len(border) == 4:
            top, bottom, left, right = border
        else:
            raise ValueError("border doit être un int ou un tuple de 2 ou 4 éléments")
    else:
        raise TypeError("border doit être un int ou un tuple")

    # Validation des dimensions
    if any(b < 0 for b in (top, bottom, left, right)):
        raise ValueError("Les bordures ne peuvent pas être négatives")
    
    if top + bottom >= height or left + right >= width:
        raise ValueError("Les bordures sont plus grandes que l'image")

    # Création du masque
    mask = np.zeros(image.shape[:2], dtype=dtype)
    mask[top:height-bottom, left:width-right] = True

    return ~mask if invert else mask

In [ ]:
def fwhm_fit(
    img: np.ndarray,
    sources: Table,
    fwhm: float,
    pixel_scale: float,
    std_lo: float = 0.5,
    std_hi: float = 0.5
) -> float:
    """
    Calcule la largeur à mi-hauteur (FWHM) d'une image en ajustant un modèle gaussien aux sources détectées.
    
    Cette fonction filtre d'abord les sources en fonction de leur flux, en gardant celles dont le flux est 
    situé dans une plage définie par les paramètres `std_lo` et `std_hi` autour de la médiane. Pour chaque 
    source filtrée, une sous-image est extraite et un ajustement d'un modèle gaussien 1D est réalisé sur la 
    distribution radiale du flux. La FWHM est alors calculée via la relation FWHM = 2.355 * stddev, convertie en 
    pixels à l'aide de `pixel_scale`.
    
    Paramètres
    ----------
    img : np.ndarray
        Image dans laquelle les sources ont été détectées.
    sources : Table
        Table contenant les positions et les flux des sources détectées. Les colonnes attendues sont :
        - 'flux' : flux total ou une mesure associée à la source.
        - 'xcentroid' : position X de la source.
        - 'ycentroid' : position Y de la source.
        - 'peak' : flux de pic de la source.
    fwhm : float
        Estimation initiale de la FWHM pour l'ajustement.
    pixel_scale : float
        Échelle en pixels pour la conversion des distances.
    std_lo : float, optionnel
        Seuil inférieur (en écart-type) pour filtrer les sources (par défaut 1.5).
    std_hi : float, optionnel
        Seuil supérieur (en écart-type) pour filtrer les sources (par défaut 2.0).
    
    Retourne
    --------
    float
        FWHM médiane estimée en pixels, basée sur l'ajustement du modèle gaussien aux sources filtrées.
    """
    try:
        # Filtrage des sources en fonction du flux
        flux = sources['flux']
        median_flux = np.median(flux)
        std_flux = np.std(flux)
        mask = (flux > median_flux + std_lo * std_flux) & (flux < median_flux + std_hi * std_flux)
        filtered_sources = sources[mask]
        
        # Suppression des sources contenant des valeurs NaN dans le flux
        filtered_sources = filtered_sources[~np.isnan(filtered_sources['flux'])]
        
        logger.info(f"Nombre de sources après filtrage: {len(filtered_sources)}")
    except Exception as e:
        logger.error(f"Erreur lors du filtrage des sources: {e}")
        raise

    # Initialisation du fitter
    fitter = fitting.SimplexLSQFitter()

    # Définition du rayon d'analyse (en pixels)
    analysis_radius = 3 * round(fwhm)
    fwhm_values = []

    # Ajustement du modèle pour chaque source filtrée
    for source in filtered_sources:
        try:
            x_cen = source['xcentroid']
            y_cen = source['ycentroid']
            peak = source['peak']

            # Définir les limites de la sous-image
            x_start, x_end = int(x_cen) - analysis_radius, int(x_cen) + analysis_radius
            y_start, y_end = int(y_cen) - analysis_radius, int(y_cen) + analysis_radius

            # Extraction de la sous-image
            sub_img = img[y_start:y_end, x_start:x_end]
            if sub_img.size == 0:
                logger.warning("Sous-image vide pour une source; passage à la suivante.")
                continue

            # Génération des grilles de coordonnées pour l'ajustement
            x_grid, y_grid = np.meshgrid(np.arange(x_start, x_end), np.arange(y_start, y_end))

            # Calcul des distances en pixels par rapport au centre de la source
            distances = np.sqrt((x_grid - x_cen) ** 2 + (y_grid - y_cen) ** 2).ravel()

            # Calcul des flux normalisés par le pic de la source
            flux_counts = (sub_img / peak).ravel()

            # Ajustement du modèle Gaussien
            gauss_model = models.Gaussian1D(amplitude=1.0, mean=0, stddev=fwhm)
            fitted_gauss = fitter(gauss_model, distances, flux_counts)

            # Calcul de la FWHM en pixels
            fwhm_gauss = 2.355 * fitted_gauss.stddev * pixel_scale
            fwhm_values.append(fwhm_gauss)
        except Exception as e:
            logger.error(f"Erreur lors de l'ajustement pour la source aux coordonnées "
                         f"({x_cen}, {y_cen}): {e}")
            continue

    if len(fwhm_values) == 0:
        msg = "Aucune source valide pour l'ajustement a été trouvée."
        logger.error(msg)
        raise ValueError(msg)

    # Conversion de la liste en array pour filtrer les NaN et valeurs infinies
    fwhm_values_arr = np.array(fwhm_values)
    # Filtrer les valeurs non numériques ou infinies
    valid = ~np.isnan(fwhm_values_arr) & ~np.isinf(fwhm_values_arr)
    if not np.any(valid):
        msg = "Toutes les valeurs de FWHM sont NaN ou infinies."
        logger.error(msg)
        raise ValueError(msg)

    mean_fwhm = np.median(fwhm_values_arr[valid])
    logger.info(f"Estimation médiane de la FWHM basée sur le modèle Gaussien: {round(mean_fwhm)} pixels")
    
    return round(mean_fwhm)

# if fitter.fit_info['final_func_val'] < 5.0:
#     color = 'green'
# else:
#     color = 'red'
    
# # Plot the data with the best-fit model
# plt.figure()
# plt.plot(pixel_distance, flux_counts, 'ko')
# rx = np.linspace(0, int(max(pixel_distance)), int(max(pixel_distance)) * 10)
# plt.plot(rx,
#          fitted_model(rx),
#          color=color,
#          lw=3.0,
#          label='SN: %.2f, Fit: %.2f, FWHM: %.2f"' % (isource['flux'] * sigma,
#                                                      fitter.fit_info['final_func_val'],
#                                                      f.value * pixel_scale.value))
# plt.xlabel('Distance (pixels)')
# plt.ylabel('Normalized flux (ADU)')
# plt.title('%s profile fit' % model)
# plt.legend()
# plt.show()

In [ ]:
@dataclass
class BackgroundStats:
    """Statistiques du fond de ciel."""
    mean: float
    median: float
    std: float
    rms: float
    global_background: float
    global_rms: float

class BackgroundResult(NamedTuple):
    """Résultats de la soustraction du fond."""
    subtracted_data: np.ndarray
    background_map: np.ndarray
    rms_map: np.ndarray
    stats: BackgroundStats
    background_obj: sep.Background

def estimate_background(
    data: np.ndarray,
    mask: Optional[np.ndarray] = None,
    box_size: Union[int, Tuple[int, int]] = 64,
    filter_size: Union[int, Tuple[int, int]] = 3,
    filter_threshold: float = 0.0,
    sigma_clip: bool = True,
    n_sigma: float = 3.0,
    return_stats: bool = True
) -> BackgroundResult:
    """
    Estime et soustrait le fond de ciel d'une image astronomique.

    Parameters
    ----------
    data : np.ndarray
        Image 2D à traiter
    mask : np.ndarray, optional
        Masque binaire (True = pixels à ignorer)
    box_size : int or tuple, optional
        Taille de la boîte pour l'estimation du fond
        - int : même taille en x et y
        - tuple : (size_y, size_x)
        (default: 64)
    filter_size : int or tuple, optional
        Taille du filtre de lissage
        - int : même taille en x et y
        - tuple : (size_y, size_x)
        (default: 3)
    filter_threshold : float, optional
        Seuil de filtrage (default: 0.0)
    sigma_clip : bool, optional
        Utiliser sigma clipping pour les statistiques (default: True)
    n_sigma : float, optional
        Nombre de sigma pour le clipping (default: 3.0)
    return_stats : bool, optional
        Calculer et retourner les statistiques détaillées (default: True)

    Returns
    -------
    BackgroundResult
        Named tuple contenant:
        - subtracted_data : Image avec fond soustrait
        - background_map : Carte du fond
        - rms_map : Carte du bruit RMS
        - stats : Statistiques du fond
        - background_obj : Objet sep.Background

    Raises
    ------
    ValueError
        Si les dimensions ou paramètres sont invalides
    TypeError
        Si les types d'entrée sont incorrects

    Notes
    -----
    - Utilise SEP pour l'estimation du fond
    - Applique optionnellement sigma clipping pour les statistiques
    - Gère automatiquement la conversion en float32 si nécessaire
    """
    # Validation des entrées
    if not isinstance(data, np.ndarray) or data.ndim != 2:
        raise ValueError("data doit être un tableau numpy 2D")
    
    # Conversion en float32 si nécessaire (requis par sep)
    if data.dtype != np.float32:
        data = data.astype(np.float32)
    
    # Gestion des tailles de boîte et de filtre
    if isinstance(box_size, int):
        bh, bw = box_size, box_size
    else:
        bh, bw = box_size
        
    if isinstance(filter_size, int):
        fh, fw = filter_size, filter_size
    else:
        fh, fw = filter_size
        
    # Validation des dimensions
    if any(s <= 0 for s in [bh, bw, fh, fw]):
        raise ValueError("Les tailles doivent être positives")
    if any(s > min(data.shape) for s in [bh, bw]):
        raise ValueError("Taille de boîte trop grande pour l'image")
        
    try:
        # Estimation du fond avec SEP
        bkg = sep.Background(
            data,
            mask=mask,
            bw=bw,
            bh=bh,
            fw=fw,
            fh=fh,
            fthresh=filter_threshold
        )
        
        # Création des cartes
        bkg_map = bkg.back()
        rms_map = bkg.rms()
        
        # Soustraction du fond
        data_sub = data - bkg_map
        
        # Calcul des statistiques détaillées si demandé
        if return_stats:
            if sigma_clip:
                mean, median, std = sigma_clipped_stats(
                    data_sub,
                    sigma=n_sigma,
                    mask=mask
                )
            else:
                if mask is not None:
                    valid_data = data_sub[~mask]
                else:
                    valid_data = data_sub
                mean = np.mean(valid_data)
                median = np.median(valid_data)
                std = np.std(valid_data)
            
            stats = BackgroundStats(
                mean=float(mean),
                median=float(median),
                std=float(std),
                rms=float(np.median(rms_map)),
                global_background=float(np.median(bkg_map)),
                global_rms=float(np.std(bkg_map))
            )
        else:
            stats = None
        
        return BackgroundResult(
            subtracted_data=data_sub,
            background_map=bkg_map,
            rms_map=rms_map,
            stats=stats,
            background_obj=bkg
        )
        
    except Exception as e:
        logger.error(f"Erreur lors de l'estimation du fond: {str(e)}")
        raise

def estimate_background_mesh(
    data: np.ndarray,
    box_size: int = 64,
    threshold: Optional[float] = None
) -> np.ndarray:
    """
    Estimation rapide du fond utilisant une approche par grille.
    Utile pour les estimations préliminaires.

    Parameters
    ----------
    data : np.ndarray
        Image 2D
    box_size : int, optional
        Taille des boîtes de la grille (default: 64)
    threshold : float, optional
        Seuil pour le rejet des sources (default: None)

    Returns
    -------
    np.ndarray
        Carte du fond estimé
    """
    h, w = data.shape
    ny = h // box_size
    nx = w // box_size
    
    # Création de la grille
    mesh = data[:ny*box_size, :nx*box_size].reshape(ny, box_size, nx, box_size)
    
    # Calcul des médianes par boîte
    if threshold is not None:
        # Avec rejet des sources
        mesh_values = np.ma.masked_greater(mesh, threshold)
        background = np.ma.median(mesh_values, axis=(1,3))
    else:
        # Sans rejet
        background = np.median(mesh, axis=(1,3))
    
    # Interpolation pour revenir à la taille originale
    from scipy.ndimage import zoom
    zoom_factor = (h/background.shape[0], w/background.shape[1])
    background = zoom(background, zoom_factor, order=1)
    
    return background

In [ ]:
def plot_2img(img1, img2, vmin, vmax, figsize=(9, 6), titles=['fig1', 'fig2'], cmap='viridis'):
    """
    Affiche deux images côte à côte avec les mêmes échelles de couleur et un titre pour chaque image.

    Paramètres :
    img1 (ndarray): La première image à afficher.
    img2 (ndarray): La deuxième image à afficher.
    vmin (np.float): Valeur minimale pour l'échelle de couleur des images.
    vmax (np.float): Valeur maximale pour l'échelle de couleur des images.
    figsize (tuple, optionnel): Taille de la figure (largeur, hauteur) en pouces (par défaut (9, 6)).
    titles (list, optionnel): Titres pour les deux images (par défaut ['fig1', 'fig2']).
    cmap (str, optionnel): Colormap à utiliser pour l'affichage des images (par défaut 'viridis').

    Retourne :
    None
    """
    # Crée une figure avec la taille spécifiée
    fig = plt.figure(figsize=figsize)

    # Affiche la première image dans le premier sous-plot
    ax1 = plt.subplot(121)
    ax1.imshow(img1, vmin=vmin, vmax=vmax, cmap=cmap, origin='lower')
    ax1.set_title(titles[0])  # Définit le titre du premier sous-plot

    # Affiche la deuxième image dans le deuxième sous-plot, partageant les axes avec le premier
    ax2 = plt.subplot(122, sharex=ax1, sharey=ax1)
    ax2.imshow(img2, vmin=vmin, vmax=vmax, cmap=cmap, origin='lower')
    ax2.set_title(titles[1])  # Définit le titre du deuxième sous-plot

    plt.tight_layout()  # Ajuste automatiquement les sous-plots pour éviter le chevauchement
    _ = plt.show()  # Affiche la figure


def pretty_print(table, precision=8):
    """
    Formate et affiche une table avec une précision numérique spécifiée.

    Paramètres :
    table (astropy.table.Table): La table à afficher.
    precision (int, optionnel): Nombre de chiffres significatifs pour les valeurs numériques (par défaut 8).

    Retourne :
    astropy.table.Table: La table formatée.
    """
    # Définit le format d'affichage pour chaque colonne de la table
    for col in table.colnames:
        table[col].info.format = '%.{}g'.format(precision)  # Formatage pour une précision uniforme
    
    # Affiche la table avec les types de données visibles
    table.pprint(show_dtype=True)
    
    return table

In [ ]:
def compute_flux(
    img: np.ndarray,
    obj: np.ndarray,
    img_sub: np.ndarray,
    mask: Optional[np.ndarray] = None,
    bg: Optional[Any] = None
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Mesure le flux et l'erreur associée pour une série d'objets détectés dans une image,
    en utilisant la méthode du rayon de Kron et une extraction par ellipse. Les objets dont
    l'aperture calculée dépasse les limites de l'image sont exclus.

    Paramètres
    ----------
    img : np.ndarray
        Image complète utilisée pour calculer le rayon de Kron.
    obj : np.ndarray
        Tableau (ou tableau structuré) contenant les paramètres des objets détectés.
        Doit contenir au minimum les colonnes suivantes :
          - 'x' : coordonnée X de l'objet.
          - 'y' : coordonnée Y de l'objet.
          - 'a' : demi-grand axe de l'ellipse.
          - 'b' : demi-petit axe de l'ellipse.
          - 'theta' : angle de rotation de l'ellipse (en radians).
    img_sub : np.ndarray
        Sous-image sur laquelle le flux est mesuré (par exemple, une région découpée de l'image).
    mask : np.ndarray, optionnel
        Masque à appliquer lors du calcul du flux, afin d'exclure certains pixels.
    bg : Any, optionnel
        Objet contenant au moins l'attribut `globalrms` pour la gestion de l'erreur (par exemple,
        le bruit de fond). Si `bg` est None, l'erreur ne sera pas prise en compte.

    Retourne
    --------
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        - flux : Les flux mesurés pour chaque objet.
        - fluxerr : Les erreurs associées aux flux mesurés.
        - flag : Les indicateurs de qualité des mesures, combinant les flags de Kron et ceux 
          provenant de `sep.sum_ellipse`.
    """
    try:
        # Calcul du rayon de Kron pour chaque objet
        kronrad, krflag = sep.kron_radius(
            img,
            obj['x'],
            obj['y'],
            obj['a'],
            obj['b'],
            obj['theta'],
            6.0
        )
    except Exception as e:
        logger.error(f"Erreur lors du calcul du rayon de Kron: {e}")
        raise

    # Gestion des valeurs de rayon de Kron négatives ou nulles
    if np.any(kronrad <= 0):
        positive = kronrad > 0
        if np.any(positive):
            replacement = np.median(kronrad[positive])
            kronrad[kronrad <= 0] = replacement
            logger.info("Kronrad ajusté avec la médiane des valeurs positives.")
        else:
            logger.error("Aucune valeur positive dans kronrad; impossible d'ajuster.")
            raise ValueError("Toutes les valeurs de kronrad sont <= 0.")

    logger.info("Kronrad ajusté.")

    # Détermination des dimensions de l'image
    image_height, image_width = img.shape

    # Vérification que les apertures restent dans les limites de l'image
    valid_aperture = (
        (obj['x'] + 2.5 * kronrad * obj['a'] < image_width) &
        (obj['x'] - 2.5 * kronrad * obj['a'] >= 0) &
        (obj['y'] + 2.5 * kronrad * obj['b'] < image_height) &
        (obj['y'] - 2.5 * kronrad * obj['b'] >= 0)
    )

    if not np.all(valid_aperture):
        logger.info("Certaines apertures sont hors des limites de l'image; elles seront exclues.")
        obj_flt = obj[valid_aperture]
        kronrad_flt = kronrad[valid_aperture]
        krflag_flt = krflag[valid_aperture]
    else:
        obj_flt = obj
        kronrad_flt = kronrad
        krflag_flt = krflag

    try:
        # Calcul du flux et des erreurs par ellipse
        flux, fluxerr, flag = sep.sum_ellipse(
            img_sub,
            obj_flt['x'],
            obj_flt['y'],
            obj_flt['a'],
            obj_flt['b'],
            obj_flt['theta'],
            2.5 * kronrad_flt,
            mask=mask,
            err=bg.globalrms if bg is not None else None
        )
    except Exception as e:
        logger.error(f"Erreur lors du calcul du flux par ellipse: {e}")
        raise

    # Combinaison des indicateurs de qualité
    flag |= krflag_flt

    return flux, fluxerr, flag

In [ ]:
def perform_psf_photometry(
    img: np.ndarray,
    phot_table: Table,
    std_x: float,
    std_y: float,
    fwhm: float,
    daostarfind: Callable
) -> Tuple[Table, np.ndarray]:
    """
    Effectue une photométrie PSF sur une image donnée.

    Cette fonction réalise la photométrie PSF en définissant un modèle PSF (ici un modèle gaussien 2D)
    et en configurant une photométrie itérative. Elle utilise les positions initiales des sources issues de
    `phot_table` et renvoie les résultats de la photométrie ainsi qu'une image résiduelle.

    Parameters
    ----------
    img : np.ndarray
        Image avec le fond de ciel soustrait.
    phot_table : astropy.table.Table
        Table contenant les positions initiales des sources. Doit contenir au moins les colonnes 
        'xcenter' et 'ycenter'.
    std_x : float
        Écart-type en x pour le modèle Gaussian2D.
    std_y : float
        Écart-type en y pour le modèle Gaussian2D.
    fwhm : float
        Full Width at Half Maximum servant à définir la taille de l'ajustement.
    daostarfind : callable
        Fonction de détection des étoiles à utiliser comme "finder" dans la photométrie PSF.

    Returns
    -------
    Tuple[astropy.table.Table, np.ndarray]
        - phot : Résultats de la photométrie PSF sous forme de table.
        - resid : Image des résidus après ajustement du modèle PSF.
    """
    try:
        # Définir le modèle PSF (ici un modèle gaussien 2D)
        model = models.Gaussian2D(amplitude=1, x_stddev=std_x, y_stddev=std_y)
        # Vous pouvez aussi utiliser un modèle de type Moffat2D en décommentant la ligne suivante :
        # model = models.Moffat2D(gamma=..., alpha=...)
        psf_model = make_psf_model(model, use_dblquad=True)
        logger.info("Modèle PSF initialisé avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'initialisation du modèle PSF: {e}")
        raise

    try:
        # Définir la forme de l'ajustement (taille de la boîte d'ajustement)
        fit_shape = 2 * round(fwhm) + 1
        logger.info(f"Forme de l'ajustement définie: {fit_shape} pixels")
    except Exception as e:
        logger.error(f"Erreur lors du calcul de la forme de l'ajustement: {e}")
        raise

    try:
        # Configurer la photométrie PSF itérative
        psfphot = IterativePSFPhotometry(
            psf_model,
            fit_shape,
            finder=daostarfind,
            aperture_radius=fit_shape / 2,
            progress_bar=True,
            fitter=fitting.LMLSQFitter()
        )
        logger.info("Photométrie PSF configurée avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de la configuration de la photométrie PSF: {e}")
        raise

    try:
        # Spécifier les positions initiales des sources à partir de la table
        psfphot.x = phot_table['xcenter']
        psfphot.y = phot_table['ycenter']
        logger.info("Positions initiales des sources définies.")
    except Exception as e:
        logger.error(f"Erreur lors de la définition des positions des sources: {e}")
        raise

    try:
        # Création d'un masque pour les pixels non finis
        mask = ~np.isfinite(img)
        # Exécuter la photométrie PSF sur l'image
        phot = psfphot(img, mask=mask)
        logger.info("Photométrie PSF exécutée avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de la photométrie PSF: {e}")
        raise

    try:
        # Calculer l'image résiduelle (différence entre l'image et le modèle PSF ajusté)
        psf_resid = psfphot.make_residual_image(img, psf_shape=(fit_shape, fit_shape))
        logger.info("Image résiduelle calculée avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors du calcul de l'image résiduelle: {e}")
        raise

    return phot, psf_resid

# # Exemple d'utilisation :
# if __name__ == "__main__":
#     # Supposons que bkg_subtracted_img, phot_table, std_x, std_y, i_fwhm et daostarfind soient définis
#     try:
#         phot, psf_resid = perform_psf_photometry(bkg_subtracted_img, phot_table, std_x, std_y, i_fwhm, daostarfind)
#         logger.info("Photométrie PSF réalisée avec succès.")
#         print(phot)
#     except Exception as e:
#         logger.error(f"Échec de la photométrie PSF: {e}")

In [ ]:
def perform_epsf_photometry(
    img: np.ndarray,
    phot_table: Table,
    fwhm: float,
    daostarfind: Any,  # On pourrait préciser Callable[..., Any] si la signature est connue
    mask: Optional[np.ndarray] = None
) -> Tuple[Table, Any]:
    """
    Effectue la photométrie PSF en utilisant un modèle EPSF.

    Parameters
    ----------
    img : np.ndarray
        Image avec le fond de ciel soustrait.
    phot_table : astropy.table.Table
        Table contenant les positions des sources. Doit contenir au moins les colonnes 'xcenter' et 'ycenter'.
    fwhm : float
        Full Width at Half Maximum servant à définir la taille de l'ajustement.
    daostarfind : callable
        Fonction de détection des étoiles utilisée comme "finder" dans la photométrie PSF.
    mask : np.ndarray, optional
        Masque pour exclure certaines zones de l'image (par exemple, pixels non finis).

    Returns
    -------
    Tuple[astropy.table.Table, photutils.epsf.EPSF]
        - phot_epsf : Résultats de la photométrie PSF avec le modèle EPSF.
        - epsf : Modèle EPSF ajusté.
    """
    try:
        # Préparer les données : conversion de l'image en objet NDData
        nddata = NDData(data=img)
        logger.info("NDData créé avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de la création de NDData: {e}")
        raise

    try:
        # Construction d'une table des étoiles à partir de phot_table
        stars_table = Table()
        stars_table['x'] = phot_table['xcenter']
        stars_table['y'] = phot_table['ycenter']
        logger.info("Table des positions d'étoiles préparée.")
    except Exception as e:
        logger.error(f"Erreur lors de la préparation de la table des étoiles: {e}")
        raise

    try:
        # Définir la forme de l'ajustement (taille de la boîte pour l'extraction des étoiles)
        fit_shape = 2 * round(fwhm) + 1
        logger.info(f"Forme d'ajustement définie: {fit_shape} pixels.")
    except Exception as e:
        logger.error(f"Erreur lors du calcul de la forme d'ajustement: {e}")
        raise

    try:
        # Extraction des étoiles dans la sous-image
        stars = extract_stars(nddata, stars_table, size=fit_shape)
        logger.info(f"{len(stars)} étoiles extraites.")
    except Exception as e:
        logger.error(f"Erreur lors de l'extraction des étoiles: {e}")
        raise

    try:
        # Affichage des étoiles extraites (optionnel)
        nrows, ncols = 5, 5
        fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5, 5), squeeze=False)
        ax = ax.ravel()
        n_disp = min(len(stars), nrows * ncols)
        for i in range(n_disp):
            norm = simple_norm(stars[i].data, 'log', percent=99.0)
            ax[i].imshow(stars[i].data, norm=norm, origin='lower', cmap='viridis')
        plt.tight_layout()
        plt.show()
        logger.info("Affichage des étoiles extraites effectué.")
    except Exception as e:
        logger.warning(f"Erreur lors de l'affichage des étoiles extraites: {e}")

    try:
        # Construction et ajustement du modèle EPSF
        epsf_builder = EPSFBuilder(oversampling=2, maxiters=5, progress_bar=True)
        epsf, fitted_stars = epsf_builder(stars)
        logger.info("Modèle EPSF ajusté avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'ajustement du modèle EPSF: {e}")
        raise

    try:
        # Affichage du modèle EPSF ajusté
        norm = simple_norm(epsf.data, 'log', percent=99.)
        plt.figure()
        plt.imshow(epsf.data, norm=norm, origin='lower', cmap='viridis', interpolation='nearest')
        plt.colorbar()
        plt.title("Modèle EPSF ajusté")
        plt.show()
        logger.info("Affichage du modèle EPSF effectué.")
    except Exception as e:
        logger.warning(f"Erreur lors de l'affichage du modèle EPSF: {e}")

    try:
        # Effectuer la photométrie PSF avec le modèle EPSF
        psfphot = IterativePSFPhotometry(
            epsf,
            fit_shape,
            finder=daostarfind,
            aperture_radius=fit_shape / 2,
            maxiters=3,
            mode='new',
            progress_bar=True
        )
        # Spécifier les positions des sources
        psfphot.x = phot_table['xcenter']
        psfphot.y = phot_table['ycenter']
        logger.info("Positions des sources définies pour la photométrie PSF.")
    except Exception as e:
        logger.error(f"Erreur lors de la configuration de la photométrie PSF: {e}")
        raise

    try:
        # Exécuter la photométrie PSF avec le masque fourni
        phot_epsf = psfphot(img, mask=mask)
        logger.info("Photométrie EPSF effectuée avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'exécution de la photométrie EPSF: {e}")
        raise

    return phot_epsf, epsf

# # Exemple d'utilisation
# if __name__ == "__main__":
#     try:
#         # Supposons que bkg_subtracted_img, phot_table, i_fwhm, daostarfind et mask soient définis
#         phot_epsf, epsf = perform_epsf_photometry(bkg_subtracted_img, phot_table, i_fwhm, daostarfind, mask)
#         logger.info("Photométrie EPSF réalisée avec succès.")
#         print(phot_epsf)
#     except Exception as e:
#         logger.error(f"Échec de la photométrie EPSF: {e}")

In [ ]:
am = airmass(header)

In [ ]:
mean, median, std, fwhm, pixel_scale = image_quality(img, img_wcs, seeing=3.0)

In [ ]:
bkg = compute_background(img, sigma=3., box_size=(50, 50), filter_size=(5, 5))
zscale_image(bkg.background)

In [ ]:
mask = make_bord_mask(img, border=100)

In [ ]:
new_img = img.astype(np.float)
img_sub, bg = sep_subtract_background(new_img, bw=50, bh=50, fw=5, fh=5, fthresh=0.01)
zscale_image(bg.back())

In [ ]:
# Convert pixel coordinates to celestial coordinates using the solved WCS
pix_coords = np.transpose((sources['xcentroid'], sources['ycentroid']))
ra_dec = im_wcs.all_pix2world(pix_coords, 0)
sources['ra'] = ra_dec[:, 0]
sources['dec'] = ra_dec[:, 1]

# Create circular apertures
radius = 1.5 * fwhm.value
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = aperture.CircularAperture(positions, 
                                      r=radius)

# Perform aperture photometry on the stars
phot_table = aperture.aperture_photometry(im_data_clean, apertures)
phot_table['ra'] = sources['ra']
phot_table['dec'] = sources['dec']

# Convert the photometry table to a pandas DataFrame
df = phot_table.copy()

In [ ]:
# Query Gaia for reference stars within the circular region
results = query_gaia(ra, dec, r.value)

In [ ]:
# Create a pandas dataframe with the reference star coordinates and magnitudes
ref_stars = pd.DataFrame({'id': results['source_id'],
                          'ra': results['ra'], 
                          'dec': results['dec'], 
                          'mag': results['phot_rp_mean_mag']})

# Convert the reference star coordinates to the same WCS as the target image
ref_coords = SkyCoord(ra=ref_stars['ra'], 
                      dec=ref_stars['dec'], 
                      unit='deg',
                      frame='icrs')
coords = ref_coords.to_table()
ref_pix_coords = im_wcs.all_world2pix(coords['ra'], coords['dec'], 0)

ref_stars['x'] = ref_pix_coords[0]
ref_stars['y'] = ref_pix_coords[1]

In [ ]:
# Match the reference stars to the detected stars
detected_stars = df[['xcenter', 'ycenter','ra','dec']]
detected_stars['aperture_sum'] = df['aperture_sum']
detected_coords = SkyCoord(ra=df['ra'], dec=df['dec'], unit='deg', frame='icrs')
idx, sep2d, dist3d = detected_coords.match_to_catalog_sky(ref_coords)
detected_stars = detected_stars.to_pandas()

match_mask = sep2d < 10*u.arcsec
matched_refs = ref_stars.iloc[idx[match_mask]]
matched_det = detected_stars.iloc[match_mask]
print(matched_refs)
print(matched_det)

In [ ]:
# Compute the photometric zeropoint
kg = 0.2 #G - 0.1 #R
m_inst = -2.5*np.log10(matched_det['aperture_sum']/60) + kg*airm
m_ref = matched_refs['mag']

dd = pd.DataFrame(columns=['mag_ref', 'mag_inst'])
dd['mag_ref'] = m_ref.values
dd['mag_inst'] = m_inst.values
dd.dropna(inplace=True)

z = np.array(dd['mag_ref'] - dd['mag_inst'])
zp = np.median(z)
zmad = scipy.stats.median_abs_deviation(z)
zero.append([zp, zmad])

# Apply the photometric zeropoint to the target stars
df['mag'] = -2.5*np.log10(df['aperture_sum']/60) + zp + kg*airm
dd['mag_inst'] = dd['mag_inst'] + zp

df = df.to_pandas()
df.sort_values(by='mag', ascending=True, inplace=True)

print(f"ZP : {zp}, ZMad : {zmad}")
print(f"Stars matched : {len(dd)}")

d = dd.drop(dd[abs(dd['mag_ref'] - dd['mag_inst']) >= 1.].index)
data_final.append(d)

print(f"Stars dropped delta >= 1. : {len(dd) - len(d)}")
print("------------------------------------------")

In [ ]:
plt.figure(figsize =(6, 6))
for i in data_final:
    sb.regplot(x=d['mag_ref'],
               y=d['mag_inst'], 
               robust=True)
plt.grid()

plt.figure(figsize =(6, 6))
for i in data_final:
    sb.residplot(x=i['mag_ref'], 
                 y=i['mag_inst'],
                 lowess=True,
                 robust=True, 
                 color='red')
plt.ylim(-0.5, 0.5)
plt.grid()

In [ ]:
p = subprocess.Popen(["C:\SAOImageDS9\ds9.exe"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

time.sleep(3)
# viewer = imexam.connect()  # startup a new DS9 window
# viewer.load_fits(fitsname)  # load a fits image into it
# viewer.scale()  # run default zscaling on the image

# Configurazione di SAMP
client = SAMPIntegratedClient()
client.connect()  # Connessione al server SAMP


# Invio del file FITS a DS9
try:
    client.notify_all({
        "samp.mtype": "image.load.fits",
        "samp.params": {
            "url": f"file://{fitsname}"
        }
    })
    
    # Impostazioni di DS9 (zscale, zoomfit, etc.)
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "zscale"
        }
    })
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "cmap viridis"
        }
    })
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "grid yes"
        }
    })
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "catalog import tsv catalogue.tsv"
        }
    })
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "zoom to fit"
        }
    })
    client.notify_all({
        "samp.mtype": "ds9.set",
        "samp.params": {
            "cmd": "smooth"
        }
    })

finally:
    client.disconnect()  # Disconnessione dal server SAMP

output, errors = p.communicate()

In [ ]:
sigma = 6.
daofind = detection.DAOStarFinder(fwhm=2.*round(fwhm), threshold=sigma*std)
sources = daofind(img - bkg.background, mask=mask)

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

# Tri des sources détectées par ordre décroissant de flux
sources.sort(keys='flux', reverse=True)

# Affichage du nombre de sources détectées
sources = pretty_print(sources, precision=8)

In [ ]:
# Bkg subtracted image
bkg_subtracted_img = img - bkg.background
i_fwhm = fwhm_fit(bkg_subtracted_img, sources, fwhm, pixel_scale)

In [ ]:
sigma = 3
daostarfind = detection.DAOStarFinder(fwhm=2.*i_fwhm, threshold=sigma*std)
final_sources = daostarfind(bkg_subtracted_img, mask=mask)

mask_valid = ~np.isnan(final_sources['xcentroid']) & ~np.isnan(final_sources['ycentroid']) & \
             ~np.isinf(final_sources['xcentroid']) & ~np.isinf(final_sources['ycentroid'])
final_sources = final_sources[mask_valid]

positions = np.transpose((final_sources['xcentroid'], final_sources['ycentroid']))

# Tri des sources détectées par ordre décroissant de flux
final_sources.sort(keys='flux', reverse=True)

# Affichage du nombre de sources détectées
final_sources = pretty_print(final_sources, precision=8)

In [ ]:
def make_source_mask(data, nsigma, npixels, filter_fwhm=3.0, dilate_size=11):
    """
    Crée un masque de sources à partir de données en utilisant une convolution et une détection de sources.

    Parameters:
    - data (numpy.ndarray): Image ou tableau de données à analyser.
    - nsigma (np.float): Seuil en multiples d'écart-type pour la détection des sources.
    - npixels (int): Nombre minimum de pixels connectés pour qu'une détection soit considérée comme une source.
    - filter_fwhm (np.float, optional): Full Width at Half Maximum (FWHM) pour le noyau gaussien utilisé dans la convolution. La valeur par défaut est 3.0.
    - dilate_size (int, optional): Taille du masque de dilatation pour agrandir le masque des sources détectées. La valeur par défaut est 11.

    Returns:
    - numpy.ndarray: Masque binaire où les sources détectées sont marquées.
    """
    
    # Créer un noyau gaussien 2D pour la convolution
    kernel = make_2dgaussian_kernel(filter_fwhm, size=3)
    
    # Convoluer les données avec le noyau gaussien
    convolved_data = convolve(data, kernel)
    
    # Appliquer une coupure sigma pour filtrer les valeurs extrêmes
    sigma_clip = SigmaClip(sigma=3.0, maxiters=5)
    clipped_data = sigma_clip(data, masked=False)
    
    # Calculer la moyenne et l'écart-type des données coupées
    mean_ = np.nanmean(clipped_data)
    std_ = np.nanstd(clipped_data)
    
    # Définir le seuil pour la détection des sources basé sur nsigma
    threshold_2sigma = mean_ + nsigma * std_
    
    # Détecter les sources dans les données convoluées
    segm = detect_sources(convolved_data, threshold_2sigma, npixels)
    
    # Créer un masque de sources avec une dilatation
    return segm.make_source_mask(size=dilate_size)

In [ ]:
def dilate_mask(mask, tophat_size):
    """
    Dilate un masque binaire en utilisant un noyau top-hat 2D.

    Cette fonction agrandit les régions du masque binaire en utilisant la convolution avec un noyau top-hat.
    Les régions dilatées sont définies comme les zones où la valeur du masque convolué dépasse un seuil 
    basé sur la taille du noyau.

    Parameters:
    - mask (numpy.ndarray): Masque binaire d'entrée à dilater. Les valeurs doivent être 0 ou 1.
    - tophat_size (np.float): Taille du noyau top-hat utilisé pour la dilatation. La taille est définie comme le diamètre du noyau.

    Returns:
    - numpy.ndarray: Masque dilaté, où les zones d'intérêt du masque d'origine sont étendues selon le noyau top-hat.
    """
    
    # Calculer la surface du noyau top-hat
    area = np.pi * tophat_size**2. 
    
    # Créer un noyau top-hat 2D avec la taille spécifiée
    kernel = Tophat2DKernel(tophat_size)
    
    # Convoluer le masque avec le noyau top-hat et appliquer le seuil
    # Les régions où la valeur convoluée dépasse 1/area sont considérées comme dilatées
    dilated_mask = convolve(mask, kernel) >= 1. / area
    
    return dilated_mask

In [ ]:
class SourceMask:
    """
    Classe pour créer et dilater un masque de sources à partir d'une image.

    Cette classe fournit des méthodes pour créer un masque de sources à différentes échelles
    et pour dilater ces masques en utilisant un noyau top-hat. Elle utilise les fonctions
    `make_source_mask` et `dilate_mask` pour ces opérations.

    Attributes:
    - img (numpy.ndarray): Image sur laquelle le masque de sources sera créé.
    - nsigma (np.float): Seuil en multiples d'écart-type pour la détection des sources.
    - npixels (int): Nombre minimum de pixels connectés pour qu'une détection soit considérée comme une source.
    """

    def __init__(self, img, nsigma=3., npixels=3):
        """
        Initialise l'objet SourceMask.

        Parameters:
        - img (numpy.ndarray): Image sur laquelle le masque sera basé.
        - nsigma (np.float, optional): Seuil en écarts-types pour la détection des sources. La valeur par défaut est 3.0.
        - npixels (int, optional): Nombre minimum de pixels connectés pour la détection. La valeur par défaut est 3.
        """
        self.img = img
        self.nsigma = nsigma
        self.npixels = npixels

    def single(self, filter_fwhm=3., tophat_size=5., mask=None):
        """
        Crée un masque de sources pour une seule échelle.

        Cette méthode crée un masque de sources à une échelle spécifiée par `filter_fwhm` 
        et dilate ce masque en utilisant un noyau top-hat de taille `tophat_size`.

        Parameters:
        - filter_fwhm (np.float, optional): Full Width at Half Maximum pour le noyau gaussien utilisé dans la détection des sources. La valeur par défaut est 3.0.
        - tophat_size (np.float, optional): Taille du noyau top-hat utilisé pour dilater le masque. La valeur par défaut est 5.0.
        - mask (numpy.ndarray, optional): Masque binaire à soustraire de l'image avant la détection. La valeur par défaut est None.

        Returns:
        - numpy.ndarray: Masque dilaté pour une seule échelle.
        """
        if mask is None:
            image = self.img
        else:
            image = self.img * (1 - mask)
        
        # Créer le masque de sources à une échelle
        mask = make_source_mask(image, nsigma=self.nsigma,
                                npixels=self.npixels,
                                dilate_size=1, filter_fwhm=filter_fwhm)
        
        # Dilater le masque créé
        return dilate_mask(mask, tophat_size)

    def multiple(self, filter_fwhm=[3.], tophat_size=[3.], mask=None):
        """
        Crée un masque de sources à plusieurs échelles.

        Cette méthode crée et dilate des masques de sources pour différentes échelles 
        spécifiées par `filter_fwhm` et `tophat_size`. Les masques à chaque échelle 
        sont combinés pour obtenir un masque final.

        Parameters:
        - filter_fwhm (list of np.float, optional): Liste des Full Width at Half Maximum pour les noyaux gaussiens utilisés dans la détection des sources. La valeur par défaut est [3.0].
        - tophat_size (list of np.float, optional): Liste des tailles des noyaux top-hat utilisés pour dilater les masques. La valeur par défaut est [3.0].
        - mask (numpy.ndarray, optional): Masque binaire à soustraire de l'image avant la détection. La valeur par défaut est None.

        Returns:
        - numpy.ndarray: Masque combiné dilaté pour toutes les échelles spécifiées.
        """
        if mask is None:
            self.mask = np.zeros(self.img.shape, dtype=bool)
        
        # Combiner les masques à différentes échelles
        for fwhm, tophat in zip(filter_fwhm, tophat_size):
            smask = self.single(filter_fwhm=fwhm, tophat_size=tophat, mask=mask)
            self.mask = self.mask | smask  # Fusionner les masques à chaque itération
        
        return self.mask

In [ ]:
def find_worst_residual_near_center(resid):
    """
    Trouve la position du pixel avec le pire résidu près du centre, en évitant les bords.

    Cette fonction recherche le pixel avec le plus grand résidu (valeur maximale) dans une région circulaire 
    centrée dans l'image, en évitant les bords de l'image. La région considérée est définie par un rayon
    proportionnel à la taille de l'image.

    Parameters:
    - resid (numpy.ndarray): Image des résidus où chaque pixel représente un résidu calculé.

    Returns:
    - tuple: Coordonnées (ligne, colonne) du pixel avec le pire résidu dans la région définie.
    """
    
    # Calculer les coordonnées du centre de l'image
    yc, xc = resid.shape[0] / 2., resid.shape[1] / 2.
    
    # Définir le rayon de la région circulaire autour du centre pour éviter les bords
    radius = resid.shape[0] / 3.
    
    # Créer des indices pour chaque pixel dans l'image
    y, x = np.mgrid[0:resid.shape[0], 0:resid.shape[1]]
    
    # Créer un masque circulaire pour la région autour du centre
    mask = np.sqrt((y - yc) ** 2 + (x - xc) ** 2) < radius
    
    # Appliquer le masque à l'image des résidus
    rmasked = resid * mask
    
    # Trouver l'indice du pixel avec la valeur maximale dans la région masquée
    return np.unravel_index(np.argmax(rmasked), resid.shape)

In [ ]:
def plot_mask(scene, bkgd, mask, zmin, zmax, worst=None, smooth=0):
    """
    Crée un graphique en trois panneaux pour visualiser le masque et son impact sur l'image.

    Les trois panneaux montrent :
    1. Le masque appliqué à l'ensemble de l'image.
    2. L'image `scene` après soustraction de l'image `bkgd` et application du masque.
    3. Une région zoomée de l'image avec le masque superposé en contours.

    Parameters:
    - scene (numpy.ndarray): Image principale sur laquelle le masque est appliqué.
    - bkgd (numpy.ndarray): Image de fond à soustraire de l'image principale.
    - mask (numpy.ndarray): Masque binaire à appliquer à l'image.
    - zmin (np.float): Valeur minimale pour l'échelle de couleurs de l'image.
    - zmax (np.float): Valeur maximale pour l'échelle de couleurs de l'image.
    - worst (tuple of int, optional): Coordonnées (x, y) du pixel avec le pire résidu à mettre en évidence. Si None, le pire résidu est déterminé automatiquement.
    - smooth (np.float, optional): Taille du noyau gaussien pour lisser l'image après soustraction. Si 0, le lissage est désactivé. La valeur par défaut est 0.

    Returns:
    - tuple: Coordonnées (x, y) du pixel avec le pire résidu, ou celles fournies par `worst` si spécifié.
    """
    
    # Déterminer les coordonnées du pire résidu
    if worst is None:
        y, x = find_worst_residual_near_center(bkgd - np.std(bkgd))
    else:
        x, y = worst
    
    # Créer une figure avec trois sous-graphes
    plt.figure(figsize=(20, 10))
    
    # Premier panneau : afficher le masque
    plt.subplot(131)
    plt.imshow(mask, vmin=0, vmax=1, cmap=plt.cm.gray, origin='lower')
    plt.title("Mask")
    
    # Deuxième panneau : afficher la scène après soustraction et application du masque
    plt.subplot(132)
    if smooth == 0:
        plt.imshow((scene - bkgd) * (1 - mask), vmin=zmin, vmax=zmax, origin='lower')
    else:
        # Appliquer un lissage gaussien si spécifié
        smoothed = convolve((scene - bkgd) * (1 - mask), Gaussian2DKernel(smooth))
        plt.imshow(smoothed * (1 - mask), vmin=zmin / smooth, vmax=zmax / smooth, origin='lower')
    plt.title("Scene with Mask Applied")
    
    # Troisième panneau : afficher la scène avec le masque en contours
    plt.subplot(133)
    plt.imshow(scene - bkgd, vmin=zmin, vmax=zmax)
    plt.contour(mask, colors='red', alpha=0.2)
    plt.title("Zoomed-In with Mask Contours")
    
    # Retourner les coordonnées du pire résidu
    return x, y

In [ ]:
mask_3sigma = make_source_mask(bkg_subtracted_img, nsigma=3, npixels=5,
                               dilate_size=5, filter_fwhm=3)

In [ ]:
zmin, zmax = np.percentile(bkg.background, (0.1, 99.9))
sm = SourceMask(bkg_subtracted_img, nsigma=1.5)
mask = sm.multiple(filter_fwhm=[1, 3, 5],
                   tophat_size=[4, 2, 1])

plot_mask(img, bkg.background, mask, zmin, zmax)

bkg2 = compute_background(img, sigma=3., box_size=(35,50), filter_size=(5,7), mask=mask_3sigma)
plot_2img(bkg.background, bkg2.background, zmin, zmax, titles=['', ''])

In [ ]:
# import numpy as np
# from astropy.io import fits
# from astropy.wcs import WCS
# from astropy.stats import SigmaClip
# from astropy.visualization import simple_norm
# from photutils import Background2D, MedianBackground
# from photutils.detection import DAOStarFinder
# from photutils.aperture import CircularAperture, aperture_photometry
# from astroquery.vizier import Vizier
# from astropy.coordinates import SkyCoord
# from astropy.table import Table, hstack
# import astropy.units as u

# # Step 1: Read the FITS file, extract data, header, and WCS information
# filename = 'HorseHead.fits'
# with fits.open(filename) as hdul:
#     data = hdul[0].data
#     header = hdul[0].header
#     wcs = WCS(header)

# # Step 2: Create a mask to exclude a border of 50 pixels around the edge of the image
# border_size = 50
# mask = np.zeros_like(data, dtype=bool)
# mask[border_size:-border_size, border_size:-border_size] = True

# # Step 3: Compute the background and remove it using a sigma clip of 3 sigma
# sigma_clip = SigmaClip(sigma=3)
# bkg_estimator = MedianBackground()
# bkg = Background2D(data, (50, 50), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator, mask=~mask)
# data_bkg_subtracted = data - bkg.background

# # Step 4: Detect sources using DAO algorithm on 3 sigma threshold
# daofind = DAOStarFinder(fwhm=1.5*i_fwhm, threshold=3.*np.median(bkg.background_rms))
# sources = daofind(data_bkg_subtracted)

# # Step 6: Compute aperture photometry on 1.5 and 2 FWHM and 2.5 radius
# positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
# apertures = [CircularAperture(positions, r=r) for r in [1.5*i_fwhm, 2*i_fwhm, 2.5]]
# phot_table = Table()

# for aperture in apertures:
#     phot = aperture_photometry(data_bkg_subtracted, aperture)
#     phot_table = hstack([phot_table, phot])

# # Step 7: Query Vizier using astroquery to search around a box with the same dimension of the image for standard sources
# vizier = Vizier(columns=['RAJ2000', 'DEJ2000', 'Vmag'])
# ra_center, dec_center = wcs.all_pix2world(data.shape[1] // 2, data.shape[0] // 2, 1)
# coord_center = SkyCoord(ra_center, dec_center, unit='deg')
# width = u.Quantity((data.shape[1], data.shape[0]), unit=u.pixel)
# search_box = width * wcs.wcs.cdelt
# result = vizier.query_region(coord_center, width=search_box, catalog='I/345/gaia2')

# # Step 8: Cross-match the detected sources and the Vizier query with a maximum distance of 5 arcsec
# std_sources = result[0]
# detected_coords = SkyCoord(sources['xcentroid'], sources['ycentroid'], frame='icrs', unit='deg', obstime='J2000')
# std_coords = SkyCoord(std_sources['RAJ2000'], std_sources['DEJ2000'], frame='icrs', unit='deg', obstime='J2000')

# idx, d2d, _ = std_coords.match_to_catalog_sky(detected_coords)
# match_mask = d2d < 5 * u.arcsec

# matched_std_sources = std_sources[match_mask]
# matched_detected_sources = sources[idx[match_mask]]

# # Step 9: Calculate the zero point of the image using these sources
# detected_fluxes = matched_detected_sources['flux']
# std_magnitudes = matched_std_sources['Vmag']
# zero_points = std_magnitudes + 2.5 * np.log10(detected_fluxes)
# zero_point = np.mean(zero_points)

# print(f"Calculated zero point of the image: {zero_point:.2f} mag")
